In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 297 µs (started: 2021-11-11 01:32:17 +00:00)


In [ ]:
pip install sentencepiece

time: 3.13 s (started: 2021-11-11 01:32:19 +00:00)


In [ ]:
import pandas as pd 
import numpy as np 
import re
import string
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, TimeDistributed, LSTM, Concatenate
from tensorflow.keras.metrics import AUC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

time: 2.76 s (started: 2021-11-11 01:32:22 +00:00)


In [ ]:
mtd_df = pd.read_csv("full_disaster_df.csv", engine = 'python', encoding = 'utf-8')
#mtd_df = pd.read_csv("multi_task_disasters.csv", engine = 'python', encoding = 'utf-8')
mtd_df[mtd_df['Disaster_or_not'].isnull() == True]
mtd_df = mtd_df.sample(frac = 1)
mtd_df.fillna("None", inplace = True)
mtd_df

,tweet_texts,Disaster_or_not,Category,source_name,tweet_id,post_time
7019,This Aerial Photo Shows What Wildfires Did to ...,1,Fire,None,None,None
10298,TONIGHT: California homes and business destroy...,1,Fire,None,None,None
10883,"Tsunami Info Stmt: M5.1 020mi NW Atka Village,...",0,None,NWS_NTWC,9.46397e+17,2017-12-28 15:05:25
3979,It took me about 2 hours to get outta this flo...,1,Flood,None,None,None
14464,#DixieFire Cont.,0,None,CAL_FIRE,1.42115e+18,2021-07-30 17:00:03
...,...,...,...,...,...,...
12840,"Signs of spring are in the weather today, with...",0,None,NWS,1.36895e+18,2021-03-08 15:23:32
7177,Kevin-Prince Boateng expected to sign for Eint...,0,None,None,None,None
12403,"Tsunami Info Stmt: M4.0 045Mi NE Fajardo, Puer...",0,None,NWS_NTWC,7.53417e+17,2016-07-14 2:34:12
10188,#news #jtimberlake JT Holds Biggest Concert in...,0,None,None,None,None


time: 271 ms (started: 2021-11-11 01:32:26 +00:00)


In [ ]:
def data_preprocess(train_df):

  def remove_emoji(text):
    emoji = re.compile("["
                        u"\U0001F600-\U0001F64F"  
                        u"\U0001F300-\U0001F5FF"  
                        u"\U0001F680-\U0001F6FF"  
                        u"\U0001F1E0-\U0001F1FF"  
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags = re.UNICODE)
    return emoji.sub(r'', text)
  
  def remove_punct(text):
    mapping = str.maketrans('', '', string.punctuation)
    return text.translate(mapping)
  
  def remove_links(text):
    text = ''.join([x for x in text if x in string.printable])
    text = re.sub(r'https?://\S+|www\.\S+', "", text)
    return text


  train_df['tweet_texts'] = train_df['tweet_texts'].apply(lambda x: remove_emoji(x))
  train_df['tweet_texts'] = train_df['tweet_texts'].apply(lambda x: remove_punct(x))
  train_df['tweet_texts'] = train_df['tweet_texts'].apply(lambda x: remove_links(x))
  train_df['tweet_texts'] = train_df['tweet_texts'].apply(lambda x: str(x).lower())
  return train_df

time: 16.7 ms (started: 2021-11-11 01:32:26 +00:00)


In [ ]:
mtd_df = data_preprocess(mtd_df)

time: 456 ms (started: 2021-11-11 01:32:26 +00:00)


In [ ]:
def bert_encode(text_inputs, tokenizer, max_length = 128):
  tokens = []
  masks = []
  segments = []

  for text in text_inputs:
    text = tokenizer.tokenize(text)
    text = text[:max_length - 2]
    bert_input = ["[CLS]"] + text + ["[SEP]"]
    padding_length = max_length - len(bert_input)
    padding = [0] * padding_length

    token = tokenizer.convert_tokens_to_ids(bert_input)
    token += padding
    mask = [1] * len(bert_input) + padding
    segment = [0] * max_length

    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

  return np.array(tokens), np.array(masks), np.array(segments)

time: 18.8 ms (started: 2021-11-11 01:32:27 +00:00)


In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

time: 11.5 s (started: 2021-11-11 01:32:27 +00:00)


In [ ]:
y1 = np.array(mtd_df['Disaster_or_not'])
# y2 = np.array(mtd_df[mtd_df['Disaster_or_not'] == 1]['Category'])
# above: take y2 only if the disaster_or_not == 1
y2 = np.array(mtd_df['Category'])

label_encoder = LabelEncoder()
y1 = label_encoder.fit_transform(y1)
classes1 = label_encoder.classes_

label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(y2)
classes2 = label_encoder.classes_

num_of_classes1 = len(classes1)
num_of_classes2 = len(classes2)


import tokenization
#new 2021 tokenization.py underfoler Twitter API
#tf.gfile = tf.io.gfile

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

#use all data to train:
train_input = bert_encode(mtd_df.tweet_texts.values, tokenizer)
train_labels_y1 = tf.keras.utils.to_categorical(y1, num_classes=num_of_classes1,dtype = 'int32')
train_labels_y2 = tf.keras.utils.to_categorical(y2, num_classes=num_of_classes2,dtype = 'int32')

time: 7.18 s (started: 2021-11-11 01:32:39 +00:00)


In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(mtd_df['tweet_texts'].values, mtd_df[['Disaster_or_not', 'Category']], test_size = 0.25)

time: 9.8 ms (started: 2021-10-26 23:32:39 +00:00)


In [ ]:
# Train and test set for model evaluation:
'''
label_encoder = LabelEncoder()
y1_train = label_encoder.fit_transform(y_train['Disaster_or_not'])
classes1 = label_encoder.classes_
y1_test = label_encoder.fit_transform(y_test['Disaster_or_not'])

label_encoder = LabelEncoder()
y2_train = label_encoder.fit_transform(y_train['Category'])
classes2 = label_encoder.classes_
y2_test = label_encoder.fit_transform(y_test['Category'])

num_of_classes1 = len(classes1)
num_of_classes2 = len(classes2)


x_train = bert_encode(x_train, tokenizer)
x_test = bert_encode(x_test, tokenizer)

y1_train = tf.keras.utils.to_categorical(y1_train, num_classes = num_of_classes1,dtype = 'int32')
y1_test = tf.keras.utils.to_categorical(y1_test, num_classes = num_of_classes1,dtype = 'int32')
y2_train = tf.keras.utils.to_categorical(y2_train, num_classes = num_of_classes2, dtype = 'int32')
y2_test = tf.keras.utils.to_categorical(y2_test, num_classes = num_of_classes2, dtype = 'int32')
'''

time: 5.95 s (started: 2021-10-26 23:32:39 +00:00)


In [ ]:
def bert_model(bert_layer, max_length = 128):

  word_ids = Input(shape = (max_length,), dtype = tf.int32, name = "input_word_ids")
  masks = Input(shape = (max_length,), dtype = tf.int32, name = "input_masks")
  segments = Input(shape = (max_length,), dtype = tf.int32, name = "input_segments")
  pooled_output, sequence_output = bert_layer([word_ids, masks, segments])
 
  #shared = Dense(64, activation = 'relu', name = "shared_bert_layer")(sequence_output[:,0,:])
  shared = Dense(64, activation = 'relu', name = "shared_bert_layer")(pooled_output)

  sub1 = Dense(32, activation = 'relu', name = "isDisaster_layer")(shared)
  sub1 = Dropout(0.2)(sub1)

  sub2 = Dense(32, activation = 'relu', name = "categories_layer")(shared)
  sub2 = Dropout(0.2)(sub2)

  out1 = Dense(num_of_classes1, activation = 'sigmoid', name = "isDisaster_layer_out")(sub1)
  out2 = Dense(num_of_classes2, activation = 'softmax', name = "categories_layer_out")(sub2)

  model = Model(inputs = [word_ids, masks, segments], outputs = [out1, out2])

  loss1 = BinaryCrossentropy()
  loss2 = CategoricalCrossentropy()

  optimizer = Adam(learning_rate = 5e-5, epsilon = 1e-8, clipnorm = 1)
  #6e-6
  model.compile(optimizer = optimizer, loss = {'isDisaster_layer_out': loss1, 'categories_layer_out': loss2}, 
                metrics = ['accuracy', AUC(from_logits=True)])
  return model

time: 31.1 ms (started: 2021-11-11 01:32:51 +00:00)


In [ ]:
model = bert_model(bert_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 input_segments (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 128, 768)]                'input_masks[0][0]',        

In [ ]:
validation_split = 0.2
epochs = 7
batch_size = 16

#full data to train:
train_history = model.fit(train_input, [train_labels_y1, train_labels_y2], validation_split = validation_split, epochs = epochs, batch_size = batch_size)

#train test set:
#train_history = model.fit(x_train, [y1_train, y2_train], validation_split = validation_split, epochs = epochs, batch_size = batch_size)

Epoch 1/7
742/742 [==============================] - 680s 879ms/step - loss: 0.9942 - isDisaster_layer_out_loss: 0.3612 - categories_layer_out_loss: 0.6330 - isDisaster_layer_out_accuracy: 0.8419 - isDisaster_layer_out_auc: 0.9199 - categories_layer_out_accuracy: 0.8089 - categories_layer_out_auc: 0.9725 - val_loss: 0.6000 - val_isDisaster_layer_out_loss: 0.2623 - val_categories_layer_out_loss: 0.3377 - val_isDisaster_layer_out_accuracy: 0.8784 - val_isDisaster_layer_out_auc: 0.9577 - val_categories_layer_out_accuracy: 0.8716 - val_categories_layer_out_auc: 0.9920
Epoch 2/7
742/742 [==============================] - 651s 878ms/step - loss: 0.5642 - isDisaster_layer_out_loss: 0.2316 - categories_layer_out_loss: 0.3326 - isDisaster_layer_out_accuracy: 0.9028 - isDisaster_layer_out_auc: 0.9671 - categories_layer_out_accuracy: 0.8878 - categories_layer_out_auc: 0.9904 - val_loss: 0.6163 - val_isDisaster_layer_out_loss: 0.2739 - val_categories_layer_out_loss: 0.3424 - val_isDisaster_layer_o

In [ ]:
pred_results = []
true_results = []
for index in range(len(x_test[0])):
  input = np.array([x_test[0][index]]), np.array([x_test[1][index]]), np.array([x_test[2][index]])
  pred = model.predict(input)
  temp_pred_res = [ int(np.argmax(pred[0], axis = 1)), int(np.argmax(pred[1], axis = 1)) ] 
  pred_results.append(temp_pred_res)
  true_results.append([ np.argmax(y1_test[index], axis = 0), np.argmax(y2_test[index], axis = 0) ])

time: 3min 7s (started: 2021-10-26 23:41:25 +00:00)


In [ ]:
y2_pre = []
y2_true = []
for item in pred_results:
  y2_pre.append(item[1])
for item in true_results:
  y2_true.append(item[1])


y1_pre = []
y1_true = []
for item in pred_results:
  y1_pre.append(item[0])
for item in true_results:
  y1_true.append(item[0])

time: 10.9 ms (started: 2021-10-26 23:51:38 +00:00)


In [ ]:
print("micro", f1_score(y2_true, y2_pre, average='micro'))
print("micro", f1_score(y1_true, y1_pre, average='micro'))

micro 0.8703504043126684
micro 0.876010781671159
time: 16.3 ms (started: 2021-10-26 23:51:53 +00:00)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

m = MultiLabelBinarizer().fit(true_results)

print("macro:", f1_score(m.transform(true_results), m.transform(pred_results), average='macro'))

print("micro", f1_score(m.transform(true_results), m.transform(pred_results), average='micro'))

macro: 0.7638653455058909
micro 0.8792685460968345
time: 40.8 ms (started: 2021-10-22 07:05:16 +00:00)


In [ ]:
#test_str = ["tornado threats from nicholas should remain low tonight across and inland of mid and upper texas coastal areas"]
test_str = ['hopkinsfire off hopkins st and north state st near calpella in mendocino county']
test_str_input = bert_encode(test_str, tokenizer)
pre = model.predict(test_str_input)
print(pre)
#pre_class = np.argmax(pre, axis = 1)
#print(pre_class)
disaster_1_0_label = np.argmax(pre[0], axis = 1)
print(classes1[disaster_1_0_label])
category_label = np.argmax(pre[1], axis = 1)
print(classes2[category_label])

[array([[0.01357793, 0.9859977 ]], dtype=float32), array([[1.3285073e-05, 1.3139224e-04, 9.9132127e-01, 1.1875490e-05,
        1.7556142e-05, 8.3728041e-03, 4.5029185e-05, 4.6102912e-05,
        2.9212797e-05, 1.1604942e-05]], dtype=float32)]
[1]
['Fire']
time: 107 ms (started: 2021-11-09 00:43:35 +00:00)


In [ ]:
model.save("bert_multi_task_1.h5")

time: 6.68 s (started: 2021-11-09 00:44:02 +00:00)


In [ ]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


time: 23.4 s (started: 2021-11-09 00:44:42 +00:00)


In [ ]:
saved_model = tf.keras.models.load_model('bert_multi_task.h5', custom_objects={"KerasLayer": hub.KerasLayer})

time: 10.6 s (started: 2021-10-15 02:55:36 +00:00)


In [ ]:
preidct = saved_model.predict(test_str_input)

time: 1.17 s (started: 2021-10-15 02:56:31 +00:00)


In [ ]:
preidct

[array([[0.00583644, 0.99205935]], dtype=float32),
 array([[1.6407538e-04, 5.3222990e-05, 9.9465513e-01, 4.1503663e-05,
         4.1551681e-04, 3.5083636e-03, 1.0112810e-03, 1.1156416e-04,
         1.4649227e-05, 2.4700434e-05]], dtype=float32)]

time: 8.61 ms (started: 2021-10-15 03:45:32 +00:00)


In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
classes1

array([0, 1])

In [ ]:
classes2

array(['Blizzard', 'Earthquake', 'Fire', 'Flood', 'Hurricane', 'None',
       'Storm', 'Tornado', 'Tsunami', 'Volcano'], dtype=object)

In [ ]:
from flask import jsonify

In [ ]:
res1 = [1, 'Tornado']
res2 = [['san', 'jose'], ['california']]
locations = [" ".join(item) for item in res2]
print(locations)



['san jose', 'california']
